In [ ]:
from google.colab import drive
drive.mount("/content/drive")

ValueError: Mountpoint must not already contain files

In [ ]:
# installing the packages
!pip install -q transformers
!pip install -q peft
!pip install -q evaluate
!pip install -q seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [ ]:
# loading dataset
from datasets import load_dataset

dataset = load_dataset("HaniehPoostchi/persian_ner")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for HaniehPoostchi/persian_ner contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/HaniehPoostchi/persian_ner.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/5121 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2560 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 5121
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2560
    })
})

In [ ]:
# based on dataset card
# "O", "I-event", "I-fac", "I-loc", "I-org", "I-pers", "I-pro", "B-event", "B-fac", "B-loc", "B-org", "B-pers", "B-pro"

In [ ]:
label_list = dataset["train"].features["ner_tags"].feature.names

label_list

['O',
 'I-event',
 'I-fac',
 'I-loc',
 'I-org',
 'I-pers',
 'I-pro',
 'B-event',
 'B-fac',
 'B-loc',
 'B-org',
 'B-pers',
 'B-pro']

In [ ]:
import datasets
import numpy as np
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification

In [ ]:
dataset['train'].description

'The dataset includes 250,015 tokens and 7,682 Persian sentences in total. It is available in 3 folds to be used in turn as training and test sets. The NER tags are in IOB format.\n'

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

In [ ]:
# testing the tokenizer on one example to see the result

example_text = dataset['train'][0]

tokenized_input = tokenizer(example_text["tokens"], is_split_into_words=True)

tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

word_ids = tokenized_input.word_ids()

print(word_ids)

''' As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to None and all other tokens to their respective word. This way, we can align the labels with the processed input ids. '''

tokenized_input

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,

{'input_ids': [2, 2031, 2339, 4281, 3043, 11084, 5966, 3655, 331, 11751, 331, 30672, 2858, 331, 2393, 2028, 6204, 4817, 300, 3001, 331, 94571, 9126, 7446, 2081, 2049, 2028, 5354, 4203, 3633, 300, 2031, 7302, 4819, 331, 17727, 3753, 4962, 2037, 2531, 4950, 9410, 331, 2725, 2036, 5860, 4425, 3473, 3467, 300, 3174, 2028, 3278, 4312, 2174, 2202, 5851, 2165, 7302, 4532, 331, 10282, 2031, 4343, 2302, 1, 2028, 12632, 7769, 1, 2882, 22034, 1158, 2263, 2045, 300, 2165, 2049, 9425, 2031, 2531, 33809, 331, 2037, 2531, 4950, 9410, 2028, 5354, 4203, 2894, 2031, 3552, 8144, 331, 2028, 2050, 2334, 2063, 2031, 2585, 4546, 2410, 333, 1157, 4793, 2050, 2983, 2028, 2690, 331, 40334, 3154, 5263, 2043, 2421, 2339, 3753, 9984, 2900, 22034, 2617, 2484, 303, 2329, 7446, 2953, 2978, 2299, 300, 14419, 2334, 3961, 3609, 2232, 9709, 1, 2353, 2165, 2049, 6370, 2224, 331, 2534, 2953, 2049, 2865, 2232, 2488, 2045, 303, 73919, 300, 10463, 2036, 7175, 5629, 7776, 2224, 300, 44495, 3171, 3081, 4235, 331, 9914, 2028, 1,

In [ ]:
len(example_text['ner_tags']), len(tokenized_input["input_ids"])

(230, 238)

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    """
    Function to tokenize and align labels with respect to the tokens. This function is specifically designed for
    Named Entity Recognition (NER) tasks where alignment of the labels is necessary after tokenization.

    Parameters:
    examples (dict): A dictionary containing the tokens and the corresponding NER tags.
                     - "tokens": list of words in a sentence.
                     - "ner_tags": list of corresponding entity tags for each word.

    label_all_tokens (bool): A flag to indicate whether all tokens should have labels.
                             If False, only the first token of a word will have a label,
                             the other tokens (subwords) corresponding to the same word will be assigned -100.

    Returns:
    tokenized_inputs (dict): A dictionary containing the tokenized inputs and the corresponding labels aligned with the tokens.
    """
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True,max_length=256, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.
        previous_word_idx = None
        label_ids = []
        # Special tokens like `<s>` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# testing the function on few example to check the result
q = tokenize_and_align_labels(dataset['train'][4:5])
print(q)

{'input_ids': [[2, 5402, 68221, 2441, 17, 2073, 2810, 2042, 7886, 1, 2613, 331, 1, 2457, 3185, 3421, 3145, 2110, 2045, 15, 4]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 11, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]}


In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q["input_ids"][0]),q["labels"][0]):
    print(f"{token:_<40} {label}")

[CLS]___________________________________ -100
حمید____________________________________ 11
طاهایی__________________________________ 5
افزود___________________________________ 0
:_______________________________________ 0
برای____________________________________ 0
اجرای___________________________________ 0
این_____________________________________ 0
طرحها___________________________________ 0
[UNK]___________________________________ 0
میلیارد_________________________________ 0
و_______________________________________ 0
[UNK]___________________________________ 0
میلیون__________________________________ 0
ریال____________________________________ 0
اعتبار__________________________________ 0
هزینه___________________________________ 0
شده_____________________________________ 0
است_____________________________________ 0
._______________________________________ 0
[SEP]___________________________________ -100


In [ ]:
# Choosing small set of dataset for increasing training speed
small_dataset = datasets.DatasetDict({
    'train': dataset['train'].shuffle(seed=42).select(range(1000)),
    'validation': dataset['test'].shuffle(seed=42).select(range(100))
})

In [ ]:
# applying function tokenize_and_align_labels on all data
tokenized_datasets = small_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets['train'][100]

{'tokens': ['این',
  'گزارش',
  'با',
  'بیان',
  'این',
  'كه',
  'علاوه',
  'بر',
  'مواد',
  'و',
  'مستندات',
  'قانونی',
  '،',
  'عرف',
  'معمول',
  'نیز',
  'مؤید',
  'حق',
  'نظارت',
  'و',
  'تحقیق',
  'و',
  'تفحص',
  'مجلس',
  'شورای',
  'اسلامی',
  'بر',
  'عملکرد',
  'قوه',
  'قضائیه',
  'است',
  '،',
  'می\u200cافزاید',
  ':',
  'از',
  'جمله',
  'این',
  'موارد',
  'می\u200cتوان',
  'از',
  'تحقیق',
  'و',
  'تفحص',
  'مجلس',
  'سوم',
  'درباره',
  'عملکرد',
  'دادستان',
  'عمومی',
  'وقت',
  'مشهد',
  '»',
  'رئیس',
  'فعلی',
  'دادگستری',
  'تهران',
  '«',
  'در',
  'جریان',
  'برخورد',
  'با',
  'کشاورزان',
  'منطقه',
  'خراسان',
  'و',
  'نیز',
  'تحقیق',
  'و',
  'تفحص',
  'مجلس',
  'دوره',
  'مذکور',
  'درباره',
  'عملکرد',
  'یکی',
  'از',
  'قضات',
  'شهرستان',
  'نهاوند',
  'نام',
  'برد',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  10,
  4,
  4,
  0,
  0,
  10,
  4,
  0

In [ ]:
label_list

['O',
 'I-event',
 'I-fac',
 'I-loc',
 'I-org',
 'I-pers',
 'I-pro',
 'B-event',
 'B-fac',
 'B-loc',
 'B-org',
 'B-pers',
 'B-pro']

In [ ]:
# Load base model
model = AutoModelForTokenClassification.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased", num_labels=len(label_list), ignore_mismatched_sizes=True)

pytorch_model.bin:   0%|          | 0.00/651M [00:00<?, ?B/s]

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-ner-uncased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-ner-uncased and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([21, 768]) in the checkpoint and torch.Size([13, 768]) in the model insta

In [ ]:
import datasets
from transformers import BertTokenizerFast, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, pipeline
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import numpy as np
import json

In [ ]:
# Configure LoRA for the model
config = LoraConfig(
    task_type="TOKEN_CLS",
    r=8,
    lora_alpha=32,
    target_modules=["classifier"],
    lora_dropout=0.1,
    bias="none",
)

# Apply LoRA to the model
model = get_peft_model(model, config)

# Training arguments
args = TrainingArguments(
    "test-ner",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

# Define compute metrics function
def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds
    pred_logits = np.argmax(pred_logits, axis=2)
    predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]
    metric = datasets.load_metric("seqeval")
    results = metric.compute(predictions=predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Initialize trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("/content/drive/MyDrive/entity/ner_model")
tokenizer.save_pretrained("/content/drive/MyDrive/entity/tokenizer")



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,2.069152,0.016279,0.082353,0.027184,0.344951
2,No log,1.504018,0.072165,0.123529,0.091106,0.877470
3,No log,1.332554,0.089147,0.135294,0.107477,0.888250


<ipython-input-28-161577c25a80>:40: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval")


The repository for seqeval contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/seqeval.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('/content/drive/MyDrive/entity/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/entity/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/entity/tokenizer/vocab.txt',
 '/content/drive/MyDrive/entity/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/entity/tokenizer/tokenizer.json')

In [ ]:
# Update the config with id2label and label2id
id2label = {str(i): label for i, label in enumerate(label_list)}
label2id = {label: str(i) for i, label in enumerate(label_list)}

config_path = "config.json"
config = json.load(open(config_path))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open(config_path, "w"))

FileNotFoundError: [Errno 2] No such file or directory: 'config.json'

In [ ]:
# Load the fine-tuned model
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/entity/ner_model")

# Run inference with the fine-tuned model
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)

# Example text for inference
example = "ابران کشور دوست داشتنی در خاورمیانه و بانک ملی است"
ner_results = nlp(example)

# Print the results
print(ner_results